In [1]:
!pip install torch torchvision matplotlib


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

print("PyTorch version:", torch.__version__)

class MNIST_CNN(nn.Module):
    def __init__(self):
        super(MNIST_CNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

PyTorch version: 2.7.1+cpu


In [3]:
batch_size = 128

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_set = torchvision.datasets.MNIST(
    root="data", train=True, download=True, transform=transform
)
test_set = torchvision.datasets.MNIST(
    root="data", train=False, download=True, transform=transform
)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

len(train_set), len(test_set)

(60000, 10000)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = MNIST_CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Using device: cpu


In [5]:
def train_one_epoch(epoch):
    model.train()
    running_loss = 0
    total = 0
    correct = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch} | Train Loss: {running_loss/total:.4f} | "
          f"Train Acc: {100*correct/total:.2f}%")


def test():
    model.eval()
    test_loss = 0
    total = 0
    correct = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Loss: {test_loss/total:.4f} | "
          f"Test Acc: {100*correct/total:.2f}%")

In [6]:
num_epochs = 5  # MNIST learns quickly!

for epoch in range(1, num_epochs + 1):
    train_one_epoch(epoch)
    test()

Epoch 1 | Train Loss: 0.1669 | Train Acc: 94.92%
Test Loss: 0.0526 | Test Acc: 98.42%
Epoch 2 | Train Loss: 0.0480 | Train Acc: 98.56%
Test Loss: 0.0401 | Test Acc: 98.68%
Epoch 3 | Train Loss: 0.0328 | Train Acc: 98.96%
Test Loss: 0.0310 | Test Acc: 98.85%
Epoch 4 | Train Loss: 0.0247 | Train Acc: 99.24%
Test Loss: 0.0331 | Test Acc: 98.97%
Epoch 5 | Train Loss: 0.0171 | Train Acc: 99.46%
Test Loss: 0.0338 | Test Acc: 98.92%


In [7]:
import os

os.makedirs("models", exist_ok=True)  # create folder if not exists
model_path = os.path.join("models", "mnist_cnn.pth")
torch.save(model.state_dict(), model_path)
print("Model saved as", model_path)

Model saved as models\mnist_cnn.pth
